## Building the Scorecard

### Loading the packages

In [1]:
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from pandas import DataFrame
from IPython.display import HTML
%matplotlib inline

In [2]:
lend = pd.read_csv('/Users/dennisvasquez/Downloads/aug2018lend.csv')

In [3]:
lend = lend.drop('Unnamed: 0', axis=1);

In [4]:
lend.head(5)

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,...,log_bc_open_to_buy,log_delinq_amnt,log_total_bal_ex_mort,log_total_il_high_credit_limit,log_annual_inc,log_tot_cur_bal,log_revol_bal,log_total_rev_hi_lim,log_total_bc_limit,status_group
0,28000,7.62%,325000.0,18.55,0,1.0,15,0,29581,54.6,...,9.539716,0.0,12.204767,12.189364,12.691580,13.591857,10.294888,10.900436,10.650176,0
1,15000,14.47%,98000.0,6.15,0,2.0,16,0,5749,22.3,...,9.619200,0.0,9.475623,8.922658,11.492723,9.475623,8.656781,10.158130,9.942708,0
2,12000,13.53%,40000.0,16.94,0,0.0,7,2,5572,68.8,...,7.264030,0.0,9.518193,9.213336,10.596635,9.518193,8.625509,8.999619,8.853665,0
3,12000,6.62%,105000.0,14.05,0,1.0,12,0,13168,21.6,...,10.582333,0.0,10.659680,10.663569,11.561716,12.497420,9.485545,11.020267,10.870471,0
4,20800,13.53%,81500.0,16.73,0,2.0,29,0,23473,54.5,...,8.826294,0.0,10.063606,0.000000,11.308358,10.063606,10.063606,10.671278,9.615805,0


In [5]:
lend.shape

(407894, 65)

In [6]:
lend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407894 entries, 0 to 407893
Data columns (total 65 columns):
loan_amnt                         407894 non-null int64
int_rate                          407894 non-null object
annual_inc                        407894 non-null float64
dti                               407894 non-null float64
delinq_2yrs                       407894 non-null int64
inq_last_6mths                    407894 non-null float64
open_acc                          407894 non-null int64
pub_rec                           407894 non-null int64
revol_bal                         407894 non-null int64
revol_util                        407894 non-null float64
total_acc                         407894 non-null int64
last_credit_pull_d                407894 non-null object
collections_12_mths_ex_med        407894 non-null float64
acc_now_delinq                    407894 non-null int64
tot_cur_bal                       407894 non-null float64
total_rev_hi_lim                  4

In [7]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [9]:
final_iv, IV = data_vars(lend , lend.status_group)

/Users/dennisvasquez/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:94: RuntimeWarning: divide by zero encountered in log
/Users/dennisvasquez/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: divide by zero encountered in log


In [10]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,loan_amnt,1000,5675,51028,11089,0.217312,39939,0.782688,0.102724,0.133154,-0.259459,0.031398
1,loan_amnt,5700,8000,51697,11377,0.220071,40320,0.779929,0.105392,0.134425,-0.243314,0.031398
2,loan_amnt,8025,10000,52004,12325,0.237001,39679,0.762999,0.114174,0.132288,-0.147252,0.031398
3,loan_amnt,10025,13000,51195,13737,0.268327,37458,0.731673,0.127255,0.124883,0.018813,0.031398
4,loan_amnt,13025,16000,55817,15310,0.274289,40507,0.725711,0.141826,0.135048,0.048972,0.031398
5,loan_amnt,16025,20000,53747,15784,0.293672,37963,0.706328,0.146217,0.126567,0.144325,0.031398
6,loan_amnt,20025,25000,43605,12905,0.295952,30700,0.704048,0.119547,0.102352,0.155292,0.031398
7,loan_amnt,25025,40000,48801,15422,0.316018,33379,0.683982,0.142864,0.111284,0.249808,0.031398
8,int_rate,10.15%,10.15%,2878,468,0.162613,2410,0.837387,0.004335,0.008035,-0.616973,0.537710
9,int_rate,10.16%,10.16%,3194,291,0.091108,2903,0.908892,0.002696,0.009678,-1.278236,0.537710


In [11]:
IV.sort_values('IV', ascending=False)

,VAR_NAME,IV
16,last_credit_pull_d,1.460082e+00
15,int_rate,5.377099e-01
11,grade_ord,3.905101e-01
10,dti,1.187544e-01
4,bc_open_to_buy,7.086619e-02
20,log_bc_open_to_buy,7.086619e-02
3,avg_cur_bal,4.865577e-02
19,log_avg_cur_bal,4.865577e-02
1,acc_open_past_24mths,4.802567e-02
58,tot_hi_cred_lim,4.434209e-02


In [12]:
mono_bin(lend.status_group,lend.tot_cur_bal,n=force_bin)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,VAR,1.0,43487.0,135966,39559,0.290948,96407,0.709052,0.366460,0.321416,0.131155,0.037024
1,VAR,43489.0,172699.0,135964,39468,0.290283,96496,0.709717,0.365617,0.321712,0.127929,0.037024
2,VAR,172700.0,4772549.0,135964,28922,0.212718,107042,0.787282,0.267923,0.356872,-0.286678,0.037024
